In [36]:
%pip install nltk transformers torch annoy seaborn matplotlib scikit-learn PyPDF2 plotly



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from annoy import AnnoyIndex
import PyPDF2
import os
import pickle
from tqdm import tqdm


nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aayushgupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aayushgupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id='PromptSystematicReview/Prompt_Systematic_Review_Dataset', 
                  repo_type='dataset', 
                  local_dir='./PapersDirectory', 
                  allow_patterns=['papers/*'], 
                  local_dir_use_symlinks=False)


Fetching 1464 files:   0%|          | 0/1464 [00:00<?, ?it/s]

'/Users/aayushgupta/Documents/GitHub/Prompt_Systematic_Review/scripts/PapersDirectory'

In [39]:
#LOADING THE FILE TEXT IN

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in pdf_reader.pages if page.extract_text() is not None])
    return text




In [40]:
#PREPROCESSING THE TEXT

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words


In [41]:
from collections import defaultdict

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Modify the embedding function for batch processing
def embed_words_batch(words):
    inputs = tokenizer(words, padding=True, return_tensors='pt', truncation=True)
    outputs = model(**inputs)
    return outputs.hidden_states[-1][:, 0, :].detach().numpy()

# Creating embeddings with caching and batch processing
def create_embedding_dictionary_batch(file_path, batch_size=10):
    document_text = read_pdf(file_path)
    words = preprocess_text(document_text)
    unique_words = list(set(words))  # Unique words for caching

    # Caching embeddings
    cached_embeddings = defaultdict(lambda: None)
    embeddings_dict = {}

    for i in range(0, len(unique_words), batch_size):
        batch_words = unique_words[i:i+batch_size]
        batch_embeddings = embed_words_batch(batch_words)

        for word, embedding in zip(batch_words, batch_embeddings):
            cached_embeddings[word] = embedding

    for word in words:
        embeddings_dict[word] = {
            'embedding': cached_embeddings[word],
            'file': file_path
        }

    return embeddings_dict


def process_multiple_pdfs(folder_path, n, batch=10):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    processed_files = all_files[:n]  # Process only the first n files

    all_embeddings = {}
    for file in tqdm(processed_files, desc="Processing PDFs"):
        file_path = os.path.join(folder_path, file)
        try:
            embeddings_dict = create_embedding_dictionary_batch(file_path, batch)
            all_embeddings.update(embeddings_dict)
        except Exception as e:  # Catching a more general exception
            print(f"Error reading PDF: {file}. Skipping this file.")
            continue

    return all_embeddings

In [42]:
def build_annoy_index(embeddings_dict):
    f = list(embeddings_dict.values())[0]['embedding'].shape[0]
    t = AnnoyIndex(f, 'angular')
    for i, (word, data) in enumerate(embeddings_dict.items()):
        t.add_item(i, data['embedding'])
    t.build(10)
    return t

def query_similar_words(query, index, embeddings_dict, top_n=5):
    query_embedding = embed_words_batch([query])[0]  # Embed the query word
    nearest_ids = index.get_nns_by_vector(query_embedding, top_n)

    similar_words_with_titles = []
    for i in nearest_ids:
        word = list(embeddings_dict.keys())[i]
        title = embeddings_dict[word]['file'].split('/')[-1]  # Extract the file name
        similar_words_with_titles.append((word, title))

    return similar_words_with_titles



In [43]:
papers_path = './PapersDirectory/papers'
num_papers_to_process = 1  # Replace with the number of papers you want to process

embeddings_dict = process_multiple_pdfs(papers_path, num_papers_to_process, 100)


Processing PDFs: 100%|██████████| 1/1 [00:07<00:00,  7.01s/it]


In [44]:
# Assuming embeddings_dict is your dictionary
with open('vector_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)
